# Загрузка библиотек

In [ ]:
!pip install optuna -q
!pip install catboost -q
!pip install natasha -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 81.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, Dataset, LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

import optuna
# from optuna.integration import CatBoostPruningCallback
from optuna.integration import CatBoostPruningCallback, LightGBMPruningCallback, XGBoostPruningCallback

import seaborn as sns
import datetime
from wordcloud import WordCloud, STOPWORDS

import pickle

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

sns.set(style="darkgrid")
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

import gensim.downloader
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.fasttext import FastText
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


фонтанка, лента, риа, дзен?

# Фиксируем значения

In [ ]:
RND_STATE = 7575
N_EST = 1000
sampler = optuna.samplers.TPESampler(seed=RND_STATE)

# Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Вероятно большая часть данных получена с сайта Лента.ру. Также видимо есть фонтанка. И возможно риа?

In [ ]:
# общий датасет собирался из трёх. Можно запустить эту часть кода и уже работать с разделом modeling
# либо собрать датасет заново (разделы ниже)
# тогда потребуется работать с новым собранным датасетом и изменить путь path_train

class Paths:
    path_train = '/content/drive/MyDrive/competitions/NLP2.0/data/parsing_data/df_5_fontanka_lenta.csv'
    path_test = '/content/drive/MyDrive/competitions/NLP2.0/data/test_news.csv'
    path_sub = '/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv'
    # path_trainw2v = '/content/drive/MyDrive/competitions/NLP2.0/tmp_data/train_prep_2.csv'
    path_prep_test = '/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/test_prep.csv'
    path_prep_train_no_ria = '/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/train_02.01_prep.csv'

paths = Paths()

In [ ]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

In [ ]:
#df_test = pd.read_csv(paths.path_test)

In [ ]:
# df_test.iloc[23103].content

'Тренер баскетбольной команды средней школы в американском штате Нью-Джерси подвергся жестокому избиению со стороны своих подопечных. Видео опубликовано на TMZ Sports. Инцидент произошел 4 февраля. Полиция отреагировала на сообщение о том, что у дверей средней школы избивают одного из ее работников. На видео четверо молодых людей бьют мужчину руками и ногами. Как утверждает полиция, нападавшие — игроки школьной баскетбольной команды. Жертвой стал их тренер. Полиция Нью-Джерси не стала разглашать имена подозреваемых. Мэр города Ньюарк пообещал, что ученики школы понесут наказание.'

## Fontanka

In [ ]:
# df_fontanka = pd.DataFrame()

# for i in range(1, 13):
#     df_ = pd.read_csv(f'https://raw.githubusercontent.com/nedokormysh/DP_NLP_bootcamp/parsing/data/fontanka_full_{i}.csv')
#     df_fontanka = pd.concat([df_fontanka, df_])

In [ ]:
# # Удаляем дубли
# df_fontanka.drop_duplicates(inplace=True)
# # Удаляем пропуски
# df_fontanka.dropna(inplace=True)
# # Переводим тэги к нижнему регистру
# df_fontanka['topic']= df_fontanka['topic'].apply(lambda x: x.lower())

# df_fontanka.shape

(26515, 6)

In [ ]:
# df_fontanka.topic.unique()

Изначальный анализ данных проводился на здесь https://github.com/nedokormysh/DP_NLP_bootcamp/tree/parsing

In [ ]:
# # di = {'Общество/Россия' : '0',
# #       'Экономика' : '1',
# #       'Силовые структуры' : '2',
# #       'Бывший СССР' : '3',
# #       'Спорт' : '4',
# #       'Забота о себе' : '5',
# #       'Строительство' : '6',
# #       'Туризм/Путешествия' : '7',
# #       'Наука и техника' : '8',
# #       }

In [ ]:
# df_fontanka = df_fontanka[df_fontanka['topic'].isin(['общество',
#                                                     #  'город',
#                                                     #  'власть',
#                                                     #  'политика',
#                                                      'финансы',
#                                                     #  'новости компаний',
#                                                      'спорт',
#                                                      'строительство',
#                                                      'недвижимость',
#                                                      'туризм',
#                                                      'технологии',
#                                                     #  'образ жизни'
#                                                      ])]

In [ ]:
# di_fontanka = {'общество' : 0,
#                'финансы' : 1,
#                'спорт' : 4,
#                'строительство' : 6,
#                'недвижимость': 6,
#                'туризм' : 7,
#                'технологии' : 8}

In [ ]:
# # Размер каждой категории
# df_fontanka.topic.value_counts()

In [ ]:
# # закодируем темы числовым кодом
# df_fontanka['topic'] = pd.Series(df_fontanka.topic.map(di_fontanka))
# df_fontanka.topic.value_counts()

0    7341
4     980
6     979
1     418
8     241
7     128
Name: topic, dtype: int64

In [ ]:
# drop_lst_f = ["url", "datetime", "views", 'title']

# df_fontanka.drop(drop_lst_f, axis=1, inplace=True)

In [ ]:
# df_fontanka.head()

## RIA

In [ ]:
# df_ria = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/tmp_data/df_ria.csv')
# df_ria_2 = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/tmp_data/df_ria_2.csv')
# df_ria2 = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/tmp_data/df_ria2.csv')
# df_ria_0 = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/tmp_data/df_ria_0.csv')

In [ ]:
# df_ria = pd.concat([df_ria, df_ria_2, df_ria2, df_ria_0])

In [ ]:
# df_ria.shape

In [ ]:
# df_ria.drop_duplicates(inplace=True)

In [ ]:
# df_ria.head(2)

In [ ]:
# drop_lst = ["id", "url", "title", "subtitle", "datetime"]

# df_ria.drop(drop_lst, axis=1, inplace=True)

In [ ]:
# df_ria.head()

In [ ]:
# di_ria = {'society': 0,
#           'economy': 1,
#           'science': 8}

In [ ]:
# df_ria['topic'] = pd.Series(df_ria.topic.map(di_ria))

## Lenta

https://github.com/nedokormysh/Stepik_Ai_edu_nlp2.0/blob/week_5_scraping_parsing/parsing_news_from_lenta_ru_.ipynb

In [ ]:
# df_tbl = pd.read_excel('/content/drive/MyDrive/competitions/NLP2.0/data/parsing_data/lenta2019-01-01_2023-12-21.xlsx')
# df_tbl.head()

,Unnamed: 0,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,0,869501,https://lenta.ru/news/2019/01/01/knife/,Трех британцев порезали ножом на вокзале,1546301460,1546301555,1,1,0,0,2,[4],https://icdn.lenta.ru/images/2019/01/01/02/20190101023935929/detail_173b9ff3938022fd4cb4edd05805847f.jpg,1546301460,Фото: @dobsonMEN Вооруженный ножом мужчина ранил трех человек на железнодорожной и трамвайной станции Виктория в Ман...,Преступника оперативно задержали,"Фото: @dobsonMEN Вооруженный ножом ... Виктория в Манчестере, Великобритания. Об этом сообщает BBC News со ссылкой н..."
1,1,869502,https://lenta.ru/news/2019/01/01/pentagon/,Пресс-секретарь Пентагона заставляла подчиненных покупать колготки и попалась,1546303800,1546353512,1,1,0,0,2,[1],https://icdn.lenta.ru/images/2019/01/01/03/20190101032720409/detail_1f1e58974c4899b26623039eb7ffb617.jpg,1546303800,Фото: Alex Brandon / AP Пресс-секретарь Пентагона Дана Уайт ушла в отставку из-за скандала с некорректным обращением...,"Она увольняла сотрудников, которые жаловались на ее поведение",Фото: Alex Brandon / AP Пресс-секретарь ... некорректным обращением с подчиненными. Прощальное письмо сотрудникам ве...
2,2,869503,https://lenta.ru/news/2019/01/01/xvii_century/,Киевских политиков обвинили в попытке вернуть страну в XVII век,1546307160,1546320399,1,1,0,0,3,[55],https://icdn.lenta.ru/images/2019/01/01/03/20190101035912755/detail_38881e2c1b6e15020025f30fa21e1a5a.jpg,1546307160,Фото: Евгений Котенко / РИА Новости Бывший президент Украины Виктор Ющенко назвал обещания киевских политиков снизит...,Ющенко сравнил публичные обещания политиков с унижением,Фото: Евгений Котенко / РИА Новости ... газ попыткой вернуть страну в XVII век. Об этом бывший президент страны Викт...
3,3,869504,https://lenta.ru/news/2019/01/01/ne_vredno/,РПЦ заступилась за «Иронию судьбы»,1546308404,1546308404,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2019/01/01/04/20190101045055141/detail_95e38179f32c5261cade1807849ca135.jpg,1546308404,"Кадр: фильм «Ирония судьбы, или С лёгким паром!» Глава синодального отдела по взаимоотношениям церкви с обществом и ...",Представитель церкви назвал фильм доброй новогодней сказкой,"Кадр: фильм «Ирония судьбы, или С лёгким ... противоречащим христианским ценностям. Об этом он написал в Telegram-ка..."
4,4,869505,https://lenta.ru/news/2019/01/01/lukashenko/,Лукашенко обошел стороной интеграцию с Россией в новогоднем обращении,1546310668,1546310668,1,1,0,0,3,[54],https://icdn.lenta.ru/images/2019/01/01/05/20190101052532644/detail_688e441ceb28eeaee76620324d80b616.jpg,1546310668,Фото: скриншот / CTVBY Президент Белоруссии Александр Лукашенко во время новогоднего обращения призвал граждан стран...,Он призвал укреплять государственность и независимость,Фото: скриншот / CTVBY Президент ... более тесной интеграции с Россией. Речь белорусского лидера опубликована на его...


In [ ]:
# df_tbl.bloc.value_counts()

In [ ]:
# # category = {1: "общество",
#                  2: "мир",
#                  3: "former_ussr",
#                  4: "economy",
#                  5: 'наука и техника',
#                  6: 'культура',
#                  7: 'интернет и сми',
#                  8: 'спорт',
#                  9: 'из жизни',
#                  12: 'среда обитания',
#                  37: 'силовые структуры',
#                  40: 'бизнес',
#                  47: 'ценности',
#                  48: 'путешествия',
#                  49:'69-парралелль',
#                  53:'нацпроекты',
#                  86:'моя страна',
#                  49:'забота о себе'}

In [ ]:
# di_lenta_tmp= {1: 'общество',
#                3: 'бывший_СССР',
#                4: 'экономика',
#                5: 'технологии',
#                8: 'спорт',
#                37: 'силовые_структуры',
#                48: 'путешествия',
#                49: 'забота_о_себе'}

In [ ]:
# df_tbl['topic'] = pd.Series(df_tbl.bloc.map(di_lenta_tmp))

In [ ]:
# df_lenta = df_tbl[df_tbl['topic'].isin(['общество',
#                                         'бывший_СССР',
#                                         'экономика',
#                                         'силовые_структуры',
#                                         'технологии',
#                                         'спорт',
#                                         'путешествия',
#                                         'забота_о_себе'])]

In [ ]:
# drop_lst = ["docid", "modified", "lastmodtime", "type", "domain", "status",
#             "part", "tags", "image_url", "rightcol", "bloc", "pubdate", "url",
#             "Unnamed: 0", "snippet",
#             'title']

# df_lenta.drop(drop_lst, axis=1, inplace=True)

In [ ]:
# di_lenta= {'общество': 0,
#            'экономика': 1,
#            'силовые_структуры': 2,
#            'бывший_СССР': 3,
#            'спорт': 4,
#            'забота_о_себе': 5,
#            'путешествия': 7,
#            'технологии': 8}

In [ ]:
# df_lenta.topic.value_counts()

In [ ]:
# df_lenta['topic'] = pd.Series(df_lenta.topic.map(di_lenta))

In [ ]:
# df_lenta.rename(columns={"text": "content"}, inplace=True)

In [ ]:
# df_lenta.head()

,content,topic
2,Фото: Евгений Котенко / РИА Новости Бывший президент Украины Виктор Ющенко назвал обещания киевских политиков снизит...,3
3,"Кадр: фильм «Ирония судьбы, или С лёгким паром!» Глава синодального отдела по взаимоотношениям церкви с обществом и ...",0
4,Фото: скриншот / CTVBY Президент Белоруссии Александр Лукашенко во время новогоднего обращения призвал граждан стран...,3
5,Фото: Михаил Маркив / РИА Новости Министр иностранных дел Украины Павел Климкин расстроился из-за большого количеств...,3
7,Фото: МЧС РФ / РИА Новости Тела еще двух погибших извлечены из-под завалов после взрыва газа и последующего обрушени...,0


In [ ]:
# df_all = pd.concat([df_fontanka,
#                     # df_ria,
#                     df_lenta])

In [ ]:
# df_all.to_csv('df_5_fontanka_lenta.csv', index=False)

# Modeling

## Уменьшение размера

In [ ]:
# def reduce_mem_usage(df):
#     """ iterate through all the columns of a dataframe and modify the data type
#         to reduce memory usage.
#     """
#     start_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

#     for col in df.columns:
#         col_type = df[col].dtype

#         if col_type != object:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)
#         # else:
#         #     df[col] = df[col].astype('category')

#     end_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
#     print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

#     return df

# def import_data(file):
#     """create a dataframe and optimize its memory usage"""
#     df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
#     df = reduce_mem_usage(df)
#     return df


# print('train')
# train = import_data(paths.path_train)

# print('-' * 80)
# print('test')
# test = import_data(paths.path_test)

train
Memory usage of dataframe is 0.61 MB
Memory usage after optimization is: 0.34 MB
Decreased by 43.7%
--------------------------------------------------------------------------------
test
Memory usage of dataframe is 0.20 MB
Memory usage after optimization is: 0.20 MB
Decreased by 0.0%


# Загрузка полностью подготовленных данных.

In [ ]:
trainw2v = pd.read_csv(paths.path_prep_train_no_ria)
test = pd.read_csv(paths.path_prep_test)

In [ ]:
# Удаляем дубли
trainw2v.drop_duplicates(inplace=True)
# Удаляем пропуски
trainw2v.dropna(inplace=True)

print(f'train shape = {trainw2v.shape},  test shape = {test.shape}')
print(f'train, test is null: {trainw2v.isna().any().any(), test.isna().any().any()}')

train shape = (39146, 3),  test shape = (26275, 2)
train, test is null: (False, False)


In [ ]:
text_features = ['content']
# cat_features_indxs = [1]
targets = ['topic']
cat_cols = ['content', 'content_clean']

filtered_features = [i for i in trainw2v.columns if (i not in targets)]

In [ ]:
# for col in cat_cols:
#     train[col] = train[col].astype('category')

In [ ]:
# train['topic'].value_counts(normalize=True)

## Разделение на фичи и целевую переменную.

In [ ]:
# на всякий случай заполним пропуски
# train = train.fillna(train.median())
# test = test.fillna(test.median())

In [ ]:
X = trainw2v[filtered_features].drop(targets, axis=1, errors="ignore")
y = trainw2v["topic"]

In [ ]:
# X.shape, y.shape

In [ ]:
# Разбиение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RND_STATE)

In [ ]:
# X_train = X_train[: 1000]
# y_train = y_train[: 1000]

## Базовая модель - catboost text features

### pipeline variant

In [ ]:
class CustomSelectColumns():
    '''
    .. notes::
    Класс для вытаскивания из датасета только требуемых колонок
    '''
    def __init__(self,
                 fil_feats: list[str]=None, clean: bool=False):
        self.fil_feats = fil_feats
        self.clean = clean

        # print('Init done')
    def fit(self, X: pd.DataFrame, y: pd.Series=None):
        return self

    def transform(self, X: pd.DataFrame, y: pd.Series=None) -> pd.DataFrame:
        X_ = X.copy()
        X_filtered = X_[self.fil_feats]
        X_filtered = pd.DataFrame(X_filtered)
        if self.clean:
            return X_filtered[self.fil_feats[0]].str.split()
        else:
            return pd.DataFrame(X_filtered)

Ниже есть эксперименты с классом Pool, и более простыми и быстрыми обработками данных и использования catboost. Но в данном случае у нас возникает проблема со стекингом. Поэтому сделал вариант через pipeline

In [ ]:
Catboost_pipe = Pipeline([
    ('col_selector', CustomSelectColumns(['content'])),
    ('Catboost_simple', CatBoostClassifier(eval_metric='Accuracy',
                           iterations=200,
                           thread_count=-1,
                           random_seed=RND_STATE,
                           text_features=text_features))
    ])

# train_dataset = Pool(data=X_train, label=y_train, text_features=['content'])
# eval_dataset = Pool(data=X_val, label=y_val, text_features=['content'])

Catboost_pipe.fit(X_train, y_train,
                  Catboost_simple__text_features=['content'],
                  Catboost_simple__verbose=5,
                  Catboost_simple__early_stopping_rounds=10,
                  Catboost_simple__eval_set=(X_val[['content']], y_val)
                  )

Learning rate set to 0.213919
0:	learn: 0.7835931	test: 0.7826309	best: 0.7826309 (0)	total: 3.49s	remaining: 11m 35s
5:	learn: 0.8351003	test: 0.8454662	best: 0.8454662 (5)	total: 20.1s	remaining: 10m 48s
10:	learn: 0.8468514	test: 0.8560664	best: 0.8560664 (10)	total: 35.8s	remaining: 10m 14s
15:	learn: 0.8543875	test: 0.8629630	best: 0.8629630 (15)	total: 51.5s	remaining: 9m 52s
20:	learn: 0.8591455	test: 0.8679438	best: 0.8679438 (20)	total: 1m 8s	remaining: 9m 43s
25:	learn: 0.8641270	test: 0.8713921	best: 0.8715198 (24)	total: 1m 23s	remaining: 9m 21s
30:	learn: 0.8675757	test: 0.8745849	best: 0.8745849 (30)	total: 1m 39s	remaining: 8m 59s
35:	learn: 0.8701622	test: 0.8766284	best: 0.8766284 (35)	total: 1m 54s	remaining: 8m 40s
40:	learn: 0.8715672	test: 0.8781609	best: 0.8781609 (40)	total: 2m 9s	remaining: 8m 20s
45:	learn: 0.8743134	test: 0.8795658	best: 0.8795658 (45)	total: 2m 24s	remaining: 8m 3s
50:	learn: 0.8767403	test: 0.8816092	best: 0.8816092 (50)	total: 2m 39s	remain

Pipeline(steps=[('col_selector',
                 <__main__.CustomSelectColumns object at 0x7ea40fb14430>),
                ('Catboost_simple',
                 <catboost.core.CatBoostClassifier object at 0x7ea40fb143a0>)])

In [ ]:
# clf_0 = CatBoostClassifier(#eval_metric='Accuracy',
#                            iterations=200,
#                            thread_count=-1,
#                            random_seed=RND_STATE,
#                            text_features=text_features)

# # train_dataset = Pool(data=X_train, label=y_train, text_features=['content'])
# # eval_dataset = Pool(data=X_val, label=y_val, text_features=['content'])

# clf_0.fit(X_train, y_train,
#           # text_features='content',
#           # verbose=50,
#           # early_stopping_rounds=200
#           )

### Pool variant

In [ ]:
# %%time
# clf_1 = CatBoostClassifier(eval_metric= 'Accuracy',
#                            iterations=200,
#                            thread_count=-1,
#                            random_seed=RND_STATE,
#                            text_features=text_features)

# train_dataset = Pool(data=X_train, label=y_train, text_features=['content'])
# eval_dataset = Pool(data=X_val, label=y_val, text_features=['content'])

# clf_1.fit(train_dataset,
#         eval_set=eval_dataset,
#         verbose=50,
#         early_stopping_rounds=200)

In [ ]:
Catboost_pipe[-1].best_score_

{'learn': {'Accuracy': 0.8977838804445012, 'MultiClass': 0.3068386418577497},
 'validation': {'Accuracy': 0.8928480204342273,
  'MultiClass': 0.32059054999907755}}

In [ ]:
# clf_1.best_score_

In [ ]:
# {'learn': {'Accuracy': 0.8775847457627118, 'MultiClass': 0.34377799294871336},
#  'validation': {'Accuracy': 0.8647572239640708,
#   'MultiClass': 0.3753871809869063}}

In [ ]:
# {'learn': {'Accuracy': 0.8886676698691979, 'MultiClass': 0.3251143718161897},
#  'validation': {'Accuracy': 0.8817483100287064,
#   'MultiClass': 0.34822207952607204}}

In [ ]:
# {'learn': {'Accuracy': 0.9008923552557616, 'MultiClass': 0.28570940502269604},
#  'validation': {'Accuracy': 0.8940415964024733,
#   'MultiClass': 0.3034087798075641}}

In [ ]:
# {'learn': {'Accuracy': 0.9285189055303997, 'MultiClass': 0.21950880724096397},
#  'validation': {'Accuracy': 0.9026315789473685,
#   'MultiClass': 0.29757526252261496}}

## Результаты базовой модели

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = Catboost_pipe.predict(test)
df_sub['Catboost_topic'] = preds
# df_sub.to_csv('try_5.csv', index=False)

In [ ]:
df_sub.head()

,topic,index,Catboost_topic
0,0,0,0.0
1,0,1,6.0
2,0,2,4.0
3,0,3,0.0
4,0,4,4.0


In [ ]:
with open(f'Catboost_pipe_model_new.pkl', "wb") as f:
        pickle.dump(Catboost_pipe, f)

### Optuna для catboost

In [ ]:
# def fit_catboost(trial, train, val):
#     X_train, y_train = train
#     X_val, y_val = val

#     param = {
#         'iterations' : 500,
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.04, step=0.001),
#         "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 30, 70, step=10),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.3, 0.4, step=0.1),
#         "auto_class_weights": trial.suggest_categorical("auto_class_weights",
#                                                         [#"SqrtBalanced",
#                                                          #"Balanced",
#                                                          "None"]),
#         "depth": trial.suggest_int("depth", 9, 10),
#         "boosting_type": trial.suggest_categorical("boosting_type", [#"Ordered",
#                                                                      "Plain"]),
#         "bootstrap_type": trial.suggest_categorical("bootstrap_type",
#                                                               [#"Bayesian",
#                                                                "Bernoulli",
#                                                                #"MVS"
#                                                                ]),
#         "used_ram_limit": "10gb",
#         "eval_metric": "Accuracy", # Тоже стоит заранее определиться
#         # 'loss_function': 'MultiClass'
#     }


#     if param["bootstrap_type"] == "Bayesian":
#         param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)

#     elif param["bootstrap_type"] == "Bernoulli":
#         param["subsample"] = trial.suggest_float("subsample", 0.1, 1)


#     clf = CatBoostClassifier(
#         **param,
#         thread_count=-1,
#         random_seed=RND_STATE,
#         text_features=text_features)
#     #  Создаем объект callback
#     pruning_callback = CatBoostPruningCallback(trial, "Accuracy")

#     clf.fit(
#         X_train,
#         y_train,
#         eval_set=(X_val, y_val),
#         verbose=0,
#         plot=False,
#         early_stopping_rounds=5,
#         callbacks=[pruning_callback],
#     )

#     # запускаем процесс прунинга
#     pruning_callback.check_pruned()

#     y_pred = clf.predict(X_val)
#     return clf, y_pred

In [ ]:
# def objective(trial, return_models=False):
#     n_splits = 3
#     kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

#     scores_simple_acc, scores_balanced_acc, models = [], [], []

#     for train_idx, valid_idx in kf.split(X_train):
#         train_data = X_train.iloc[train_idx, :], y_train.iloc[train_idx]
#         valid_data = X_train.iloc[valid_idx, :], y_train.iloc[valid_idx]

#         # Подаем trials для перебора
#         model, y_pred = fit_catboost(trial, train_data, valid_data) # Определили выше

#         # scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
#         scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
#         models.append(model)
#         break

#     result = np.mean(scores_balanced_acc)
#     # result = scores_balanced_acc
#     print(f'result = {result}')

#     if return_models:
#         return result, models
#     else:
#         return result

In [ ]:
from optuna.integration import CatBoostPruningCallback, LightGBMPruningCallback, XGBoostPruningCallback
# from optuna.integration.pytorch_lightning import PyTorchLightningPruningCallback
# optuna.logging.set_verbosity(optuna.logging.WARNING)
# # optuna.logging.set_verbosity(optuna.logging.ERROR)

In [ ]:
# study = optuna.create_study(direction="maximize",
#                             pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
#                             study_name="Catboost_clf_simple",
#                             sampler=sampler)
# study.optimize(objective,
#                n_trials=40,
#                n_jobs=-1,
#                show_progress_bar=True)

In [ ]:
# print("Number of finished trials: {}".format(len(study.trials)))
# print("Best trial:")
# trial = study.best_trial
# print("  Value: {}".format(trial.value))
# print("  Params: ")

# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [ ]:
# valid_scores, models = objective(
#     optuna.trial.FixedTrial(study.best_params),
#     return_models=True,
# )

In [ ]:
# # История изменения от числа испытаний
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# params = ['learning_rate',
#           'l2_leaf_reg',
#           'colsample_bylevel',
#           'auto_class_weights',
#           'depth',
#           'bootstrap_type',
#           'boosting_type']

# optuna.visualization.plot_slice(study, params=params, target_name='accuracy')

## Базовая модель после optuna

### pipeline variant

In [ ]:
Catboost_pipe = Pipeline([
    ('col_selector', CustomSelectColumns(['content'])),
    ('Catboost_simple', CatBoostClassifier(depth=9,
                           l2_leaf_reg=50,
                           learning_rate=0.018,
                           colsample_bylevel=0.3,
                           auto_class_weights='None',
                           bootstrap_type= 'Bernoulli',
                           boosting_type="Plain",
                           eval_metric= 'Accuracy',
                           iterations=500,
                           thread_count=-1,
                           random_seed=RND_STATE,
                           text_features=text_features))
    ])

Catboost_pipe.fit(X_train, y_train,
                  Catboost_simple__text_features=['content'],
                  Catboost_simple__verbose=50,
                  Catboost_simple__early_stopping_rounds=200,
                  Catboost_simple__eval_set=(X_val[['content']], y_val)
                  )

0:	learn: 0.7855508	test: 0.7911017	best: 0.7911017 (0)	total: 21.7s	remaining: 3h 8s
50:	learn: 0.8118220	test: 0.8213559	best: 0.8213559 (47)	total: 16m 10s	remaining: 2h 22m 23s
100:	learn: 0.8156780	test: 0.8262712	best: 0.8266102 (96)	total: 32m 16s	remaining: 2h 7m 30s
150:	learn: 0.8198305	test: 0.8290678	best: 0.8291525 (149)	total: 47m 45s	remaining: 1h 50m 24s
200:	learn: 0.8235593	test: 0.8314407	best: 0.8314407 (197)	total: 1h 3m 19s	remaining: 1h 34m 12s
250:	learn: 0.8257627	test: 0.8338983	best: 0.8338983 (249)	total: 1h 18m 50s	remaining: 1h 18m 12s
300:	learn: 0.8276271	test: 0.8358475	best: 0.8358475 (300)	total: 1h 34m 18s	remaining: 1h 2m 20s
350:	learn: 0.8301695	test: 0.8376271	best: 0.8377119 (346)	total: 1h 49m 36s	remaining: 46m 31s
400:	learn: 0.8328178	test: 0.8405085	best: 0.8405085 (399)	total: 2h 5m 12s	remaining: 30m 54s
450:	learn: 0.8348729	test: 0.8428814	best: 0.8428814 (449)	total: 2h 20m 57s	remaining: 15m 18s
499:	learn: 0.8368008	test: 0.8442373	b

Pipeline(steps=[('col_selector',
                 <__main__.CustomSelectColumns object at 0x7ba724e14f10>),
                ('Catboost_simple',
                 <catboost.core.CatBoostClassifier object at 0x7ba724e14a90>)])

In [ ]:
Catboost_pipe[-1].best_score_

{'learn': {'Accuracy': 0.8368008474576272, 'MultiClass': 0.4979845860401242},
 'validation': {'Accuracy': 0.844406779661017,
  'MultiClass': 0.4798322354607637}}

In [ ]:
preds = Catboost_pipe.predict(test)
df_sub['Catboost_topic'] = preds

In [ ]:
with open(f'Catboost_pipe_model_opt.pkl', "wb") as f:
        pickle.dump(Catboost_pipe, f)

### pool variant

In [ ]:
# %%time
# clf_2 = CatBoostClassifier(depth=9,
#                            l2_leaf_reg=50,
#                            learning_rate=0.018,
#                            colsample_bylevel=0.3,
#                            auto_class_weights='None',
#                            bootstrap_type= 'Bernoulli',
#                            boosting_type="Plain",
#                            eval_metric= 'Accuracy',
#                            iterations=500,
#                            thread_count=-1,
#                            random_seed=RND_STATE,
#                            text_features=text_features
#       )


# train_dataset = Pool(data=X_train, label=y_train, text_features=['content'])
# eval_dataset = Pool(data=X_val, label=y_val, text_features=['content'])

# clf_2.fit(train_dataset,
#         eval_set=eval_dataset,
#         verbose=10,
#         early_stopping_rounds=5)

In [ ]:
# clf_2.best_score_

In [ ]:
# {'learn': {'Accuracy': 0.8531127037661608, 'MultiClass': 0.4443919425452108},
#  'validation': {'Accuracy': 0.8643901068015739,
#   'MultiClass': 0.40232220505661986}}

In [ ]:
# df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
# preds = clf_1.predict(test)
# df_sub['topic'] = preds
# df_sub.to_csv('try_2.csv', index=False)

## Simple models

### Предобработка текстов.

In [ ]:
# segmenter = Segmenter()
# morph_vocab = MorphVocab()
# emb = NewsEmbedding()
# morph_tagger = NewsMorphTagger(emb)

In [ ]:
# stop_words = stopwords.words('russian')
# stop_words.extend(['что', 'это', 'так',
#                     'вот', 'быть', 'как',
#                     'в', '—', 'к', 'за', 'из', 'из-за',
#                     'на', 'ок', 'кстати',
#                     'который', 'мочь', 'весь',
#                     'еще', 'также', 'свой',
#                     'ещё', 'самый', 'ул', 'комментарий',
#                     'английский', 'язык'])

Функция для обработки текстов.

In [ ]:
# def text_prep(text) -> str:
#     doc = Doc(text)
#     doc.segment(segmenter)
#     doc.tag_morph(morph_tagger)

#     for token in doc.tokens:
#         token.lemmatize(morph_vocab)

#     lemmas = [_.lemma for _ in doc.tokens]
#     words = [lemma for lemma in lemmas if lemma.isalpha() and len(lemma) > 2]
#     filtered_words = [word for word in words if word not in stop_words]
#     return " ".join(filtered_words)

Обработаем тексты обучения и теста.

In [ ]:
# # Удаляем дубли
# train.drop_duplicates(inplace=True)
# # Удаляем пропуски
# train.dropna(inplace=True)

# print(f'train shape = {train.shape},  test shape = {test.shape}')
# print(f'train, test is null: {train.isna().any().any(), test.isna().any().any()}')

train shape = (39146, 2),  test shape = (26275, 1)
train, test is null: (False, False)


In [ ]:
# %%time
# train['content_clean'] = train.content.apply(text_prep)

CPU times: user 39min 56s, sys: 23min 45s, total: 1h 3min 42s
Wall time: 38min 41s


In [ ]:
# train.to_csv('train_02.01_prep.csv', index=False)

In [ ]:
# %%time
# test['content_clean'] = test.content.apply(text_prep)

## Загрузка датасетов, после предобработки

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X.content_clean.str.split(),
#                                                   y,#.values,
#                                                   test_size=0.2,
#                                                   random_state=RND_STATE)

## TFIDF

In [ ]:
# Для получения вектора предложения используем tf_idf
class TfidfEmbeddingVectorizer(object):
    """Get tfidf weighted vectors"""
    def __init__(self, model):
        self.word2vec = model.wv
        self.word2weight = None
        self.dim = model.vector_size

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec.get_vector(w) * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

## Word2Vec

In [ ]:
# # X_train, X_val, y_train, y_val = train_test_split(X.content_clean.str.split(),
# #                                                   y,#.values,
# #                                                   test_size=0.2,
# #                                                   random_state=RND_STATE)


# model_w2v = Word2Vec(sentences=X_train,
#                  vector_size=300,
#                  min_count=5,
#                  window=25,
#                  seed=RND_STATE)

In [ ]:
# with open(f'model_w2v.pkl', "wb") as f:
#         pickle.dump(model_w2v, f)

In [ ]:
with open(f"/content/drive/MyDrive/competitions/NLP2.0/models/model_w2v.pkl", "rb") as f:
        model_w2v = pickle.load(f)

## Логистическая регрессия

### W2V

In [ ]:
# pipeline variant
%%time
pipe_log = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                 ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))])

pipe_log.fit(X_train, y_train)

print(classification_report(y_val, pipe_log.predict(X_val)))

              precision    recall  f1-score   support

           0       0.84      0.87      0.85      3374
           1       0.81      0.83      0.82       924
           2       0.85      0.79      0.82       568
           3       0.83      0.82      0.83       938
           4       0.97      0.96      0.97       835
           5       0.59      0.48      0.53        46
           6       0.93      0.95      0.94       193
           7       0.87      0.79      0.83       412
           8       0.87      0.80      0.83       540

    accuracy                           0.85      7830
   macro avg       0.84      0.81      0.82      7830
weighted avg       0.85      0.85      0.85      7830

CPU times: user 3min 40s, sys: 24.3 s, total: 4min 5s
Wall time: 3min 10s


In [ ]:
# simple variant
# %%time
# pipe_log = Pipeline([('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
#                  ('scaler', StandardScaler()),
#                  ('clf', LogisticRegression(random_state=RND_STATE,
#                                             max_iter=10000,
#                                             # class_weight='balanced'
#                                             ))])

# pipe_log.fit(X_train, y_train)

# print(classification_report(y_val, pipe_log.predict(X_val)))

In [ ]:
# precision    recall  f1-score   support

#          0.0       0.78      0.82      0.80      4287
#          1.0       0.77      0.77      0.77      2110
#          2.0       0.77      0.68      0.72       981
#          3.0       0.85      0.81      0.83      1264
#          4.0       0.97      0.97      0.97      1321
#          5.0       0.60      0.47      0.53        87
#          6.0       0.95      0.96      0.95       203
#          7.0       0.87      0.83      0.85       612
#          8.0       0.80      0.78      0.79       935

#     accuracy                           0.81     11800
#    macro avg       0.82      0.79      0.80     11800
# weighted avg       0.81      0.81      0.81     11800

# CPU times: user 7min 54s, sys: 49.5 s, total: 8min 43s

### Optuna для лог_рег

In [ ]:
def fit_log_reg(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {"dual": False,
             "C": trial.suggest_float("C", 2.5, 3, step=0.1),
             "solver": trial.suggest_categorical("solver",
                                                 ["sag",
                                                  #"saga",
                                                  "lbfgs"
                                                  ]),
             "random_state": RND_STATE,
             "max_iter": trial.suggest_int("max_iter", 5500, 6300, step=200),
             'multi_class': trial.suggest_categorical("multi_class",
                                                      [#'auto',
                                                       'ovr',
                                                       #'multinomial'
                                                       ]),
             'n_jobs': -1,
             'verbose': 0
    }

    # print(f'param["solver"] = {param["solver"]}')
    if param["solver"] == "saga":
        param["penalty"] = trial.suggest_categorical("penalty",
                                             ['l1',
                                              'l2', #'elasticnet',
                                              None])
    else:
         param["penalty"] = trial.suggest_categorical("penalty",
                                             ['l2',
                                              #None
                                              ])

    clf = LogisticRegression(**param)

    pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(model_w2v)),
                    ('scaler', StandardScaler()),
                    ('clf', clf)])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_val)
    return pipe, y_pred

In [ ]:
def objective_log(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

    scores_simple_acc, scores_balanced_acc, models = [], [], []

    for train_idx, valid_idx in kf.split(X_train.index):
        train_data = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_log_reg(trial, train_data, valid_data)

        # scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
        scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break

    result = np.mean(scores_balanced_acc)
    # print(f'result = {result}')

    if return_models:
        return result, models
    else:
        return result

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            sampler=sampler,
                            study_name="Log_Reg_clf")
study.optimize(objective_log,
               n_trials=10,
               n_jobs=-1,
               show_progress_bar=True)

[I 2024-01-02 15:41:48,183] A new study created in memory with name: Log_Reg_clf


  0%|          | 0/10 [00:00<?, ?it/s]

[W 2024-01-02 16:04:47,776] Trial 0 failed with parameters: {'C': 2.3, 'solver': 'lbfgs', 'max_iter': 9010, 'multi_class': 'auto', 'penalty': None} because of the following error: KeyboardInterrupt().
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py", line 463, in _process_worker
    r = call_item()
  File "/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py", line 291, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 589, in __call__
    return [func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 589, in <listcomp>
    return [func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py", line 123, in __call__
    return self.function(*args, **kwargs)
  File "/

KeyboardInterrupt: ignored

## SVM

### W2V

In [ ]:
#pipeline version
%%time
pipe_svc = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                     ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                     ('scaler', StandardScaler()),
                     ('clf', SVC(random_state=RND_STATE,
                             max_iter=10000,
                            #  class_weight='balanced'
                             ))])

pipe_svc.fit(X_train, y_train)

print(classification_report(y_val, pipe_svc.predict(X_val)))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88      3374
           1       0.83      0.87      0.85       924
           2       0.86      0.81      0.84       568
           3       0.86      0.86      0.86       938
           4       0.97      0.98      0.97       835
           5       0.76      0.35      0.48        46
           6       0.96      0.92      0.94       193
           7       0.90      0.86      0.88       412
           8       0.90      0.82      0.86       540

    accuracy                           0.88      7830
   macro avg       0.88      0.82      0.84      7830
weighted avg       0.88      0.88      0.88      7830

CPU times: user 4min 4s, sys: 1.31 s, total: 4min 5s
Wall time: 4min 8s


### Optuna для svm

In [ ]:
def fit_svm(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {"C": trial.suggest_float("C", 1, 1.5, step=0.5),
             "kernel": trial.suggest_categorical("kernel",
                                                 [#"linear",
                                                  "poly",
                                                  #"rbf",
                                                  #'sigmoid'
                                                  ]),
             "random_state": RND_STATE,
             "degree": trial.suggest_int("degree", 3, 4, step=1),
             "max_iter": trial.suggest_int("max_iter", 5000, 6000, step=1000),
             'decision_function_shape': trial.suggest_categorical("decision_function_shape",
                                                      ['ovo',
                                                      #  'ovr'
                                                       ]),
             'verbose': 0
    }

    clf = SVC(**param)

    pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(model_w2v)),
                    ('scaler', StandardScaler()),
                    ('clf', clf)])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_val)
    return pipe, y_pred

In [ ]:
def objective_svm(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

    scores_simple_acc, scores_balanced_acc, models = [], [], []

    for train_idx, valid_idx in kf.split(X_train.index):
        train_data = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_svm(trial, train_data, valid_data)

        # scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
        scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break

    result = np.mean(scores_balanced_acc)

    if return_models:
        return result, models
    else:
        return result

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            sampler=sampler,
                            study_name="SVM_clf")
study.optimize(objective_svm,
               n_trials=4,
               n_jobs=-1,
               show_progress_bar=True)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# # История изменения от числа испытаний
# optuna.visualization.plot_optimization_history(study)

## XGboost

In [ ]:
%%time
XGboost_pipe = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                     ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                    #  ('scaler', StandardScaler()),
                     ('clf', XGBClassifier(random_state=RND_STATE,
                            #  max_iter=10000,
                            #  class_weight='balanced'
                             ))])

XGboost_pipe.fit(X_train, y_train,
                #  clf__eval_set=(X_val, y_val)
                 )

print(classification_report(y_val, XGboost_pipe.predict(X_val)))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      3374
           1       0.82      0.85      0.84       924
           2       0.85      0.77      0.81       568
           3       0.84      0.84      0.84       938
           4       0.97      0.96      0.97       835
           5       0.83      0.33      0.47        46
           6       0.96      0.90      0.93       193
           7       0.91      0.83      0.87       412
           8       0.90      0.81      0.85       540

    accuracy                           0.86      7830
   macro avg       0.88      0.80      0.83      7830
weighted avg       0.87      0.86      0.86      7830

CPU times: user 6min 6s, sys: 921 ms, total: 6min 7s
Wall time: 5min 24s


In [ ]:
# precision    recall  f1-score   support

#            0       0.85      0.89      0.87      3374
#            1       0.82      0.85      0.84       924
#            2       0.85      0.77      0.81       568
#            3       0.84      0.84      0.84       938
#            4       0.97      0.96      0.97       835
#            5       0.83      0.33      0.47        46
#            6       0.96      0.90      0.93       193
#            7       0.91      0.83      0.87       412
#            8       0.90      0.81      0.85       540

#     accuracy                           0.86      7830
#    macro avg       0.88      0.80      0.83      7830
# weighted avg       0.87      0.86      0.86      7830

# CPU times: user 5min 25s, sys: 1.74 s, total: 5min 26s
# Wall time: 3min 55s

### Optuna для XGboost

In [ ]:
def fit_xgboost(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {
        'iterations' : 500,
        'objective': 'multi:softmax',
        "booster": trial.suggest_categorical("booster", ["gbtree",
                                                         "gblinear",
                                                         "dart"
                                                             ]),
        # 'max_leaves': trial.suggest_int('max_leaves', 6, 12),
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'lambda': trial.suggest_int("lambda", 2, 15),
        'max_bin': trial.suggest_int('max_bin', 220, 350, step=10),
        'alpha': trial.suggest_int('alpha', 2, 15),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.6, 0.9, step=0.01),
        'subsample': trial.suggest_float("subsample", 0.8, 0.95, step=0.01),
        'min_child_weight': trial.suggest_float("min_child_weight", 8, 12, step=0.01),
        # "eta": trial.suggest_float("eta", 0.01, 0.04, step=0.001),
        # "used_ram_limit": "10gb",
        "eval_metric": "merror"
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
            param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
            param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
            param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
            param["tree_method"]: trial.suggest_categorical("tree_method", ['hist', 'approx'],)
    if param["booster"] == "dart":
            param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
            param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    clf = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                     ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                     ('scaler', StandardScaler()),
                     ('clf', XGBClassifier(**param,
                                           random_state=RND_STATE,
                                           n_jobs=-1
                                           ))])
    #  Создаем объект callback
    pruning_callback = XGBoostPruningCallback(trial, 'validation_0-merror')

    clf.fit(X_train, y_train,
                    #  clf__eval_set=(X_val, y_val),
                     )

    clf.fit(
        X_train,
        y_train,
        # clf__eval_set=(X_val, y_val),
        clf__verbose=0,
        # clf__early_stopping_rounds=5,
        clf__callbacks=[pruning_callback],
    )

    y_pred = clf.predict(X_val)
    return clf, y_pred

In [ ]:
def objective_xgboost(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

    scores_simple_acc, scores_balanced_acc, models = [], [], []

    for train_idx, valid_idx in kf.split(X_train.index):
        train_data = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_xgboost(trial, train_data, valid_data)

        scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
        # scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break

    result = np.mean(scores_simple_acc)

    if return_models:
        return result, models
    else:
        return result

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            sampler=sampler,
                            study_name="XGboost_clf")
study.optimize(objective_xgboost,
               n_trials=4,
               n_jobs=-1,
               show_progress_bar=True)

[I 2024-01-03 15:46:54,402] A new study created in memory with name: XGboost_clf


  0%|          | 0/4 [00:00<?, ?it/s]

[W 2024-01-03 16:13:29,767] Trial 1 failed with parameters: {'booster': 'gbtree', 'max_depth': 7, 'lambda': 12, 'max_bin': 270, 'alpha': 9, 'colsample_bytree': 0.6599999999999999, 'subsample': 0.8500000000000001, 'min_child_weight': 9.41, 'eta': 0.21027432186871034, 'gamma': 0.0064328524600991985, 'grow_policy': 'depthwise'} because of the following error: KeyError('validation_0-merror').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-59-b55717a19704>", line 12, in objective_xgboost
    model, y_pred = fit_xgboost(trial, train_data, valid_data)
  File "<ipython-input-61-0717393b612c>", line 51, in fit_xgboost
    clf.fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 730, i

KeyboardInterrupt: ignored

# Stack

In [ ]:
# Некоторые модели для построение ансамбля
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier

In [ ]:
Catboost_pipe = Pipeline([
    ('col_selector', CustomSelectColumns(['content'])),
    ('Catboost_simple', CatBoostClassifier(eval_metric='Accuracy',
                           iterations=200,
                           thread_count=-1,
                           random_seed=RND_STATE,
                           text_features=text_features))
    ])

In [ ]:
# список базовых моделей
estimators = [("CatBoost", Catboost_pipe),
              ('SVC', pipe_svc),
              ('Log_Reg', pipe_log)]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False, max_iter=200),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier = meta_model
stacking_classifier

StackingClassifier(estimators=[('CatBoost',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2af34df6a0>),
                                                ('Catboost_simple',
                                                 <catboost.core.CatBoostClassifier object at 0x7e2af34dfc70>)])),
                               ('SVC',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2ae6042770>),
                                                ('w2v_tfidf',
                                                 <__main__.TfidfEmbeddingVectorizer o...
                                                     random_state=7575))])),
                               ('Log_Reg',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2af34ded70>),
                                                ('w2v_tfidf',
                                                 <__main__.TfidfEmbeddingVectorizer object at 0x7e2af34df610>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 LogisticRegression(max_iter=10000,
                                                                    random_state=7575))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [ ]:
%%time
stacking_classifier.fit(X_train, y_train)

In [ ]:
preds_st = stacking_classifier.predict(X_val)
balanced_accuracy_score(y_true=y_val, y_pred=preds_st)

0.854229941129818

In [ ]:
accuracy_score(y_true=y_val, y_pred=preds_st)

0.9001277139208174

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = stacking_classifier.predict(test)
df_sub['topic'] = preds
df_sub.to_csv('try_6.csv', index=False)